In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [2]:
data = pd.read_csv(r"digit-recognizer\train.csv")

In [3]:
data

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41996,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41997,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41998,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
def ReLU(Z):
    return np.maximum(Z, 0)

def softmax(Z):
    Z_stable = Z - np.max(Z, axis=0, keepdims=True)
    Z_stable = np.clip(Z_stable,-1e10,1e10)
    exp_Z = np.exp(Z_stable)  # Stabilize by subtracting max
    return exp_Z / np.sum(exp_Z, axis=0, keepdims=True)

def ReLU_deriv(Z):
    return Z > 0

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def compute_loss(A3, Y):
    one_hot_Y = one_hot(Y)
    m = Y.size
    loss = -np.sum(one_hot_Y * np.log(A3)) / m
    return loss


In [5]:
def conv2D(images, filters):
    # Get shapes of images and filters
    batch_size, image_height, image_width, image_depth = images.shape
    num_filters, filter_height, filter_width, filter_depth = filters.shape
    
    # Validate that the filter depth matches the image depth
    if image_depth != filter_depth:
        raise ValueError("The depth of the images and filters must match")

    # Calculate output dimensions
    output_height = image_height - filter_height + 1
    output_width = image_width - filter_width + 1

    # Prepare an empty output array
    output = np.zeros((batch_size, output_height, output_width, num_filters))

    # Perform the convolution operation
    for b in range(batch_size):  # Iterate over batch
        for f in range(num_filters):  # Iterate over filters
            for y in range(output_height):
                for x in range(output_width):
                    # Define the region of the image being processed
                    region = images[b, y:y+filter_height, x:x+filter_width, :]
                    
                    # Perform the convolution: element-wise multiplication and sum
                    output[b, y, x, f] = np.sum(region * filters[f, :, :, :])

    return output


In [6]:
def MaxPool(images, pool_size, stride=2):
    batch_size, image_height, image_width, image_depth = images.shape
    pool_height, pool_width = pool_size

    output_height = (image_height - pool_height) // stride + 1
    output_width = (image_width - pool_width) // stride + 1

    if output_height <= 0 or output_width <= 0:
        raise ValueError("Pooling window or stride is too large for the given image dimensions.")

    output = np.zeros((batch_size, output_height, output_width, image_depth))

    for b in range(batch_size):  # Iterate over batch
        for d in range(image_depth):  # Iterate over depth
            for y in range(output_height):
                for x in range(output_width):
                    start_y = y * stride
                    start_x = x * stride
                    end_y = start_y + pool_height
                    end_x = start_x + pool_width

                    pooling_window = images[b, start_y:end_y, start_x:end_x, d]
                    output[b, y, x, d] = np.max(pooling_window)

    return output


In [7]:
def generate_batches(X, Y, batch_size=64):
    num_samples = X.shape[0]
    indices = np.random.permutation(num_samples)  # Shuffle indices
    X = X[indices]
    Y = Y[indices]

    for i in range(0, num_samples, batch_size):
        X_batch = X[i:i + batch_size]
        Y_batch = Y[i:i + batch_size]
        yield X_batch, Y_batch

def init_params():
    # Convolutional layers
    W1 = np.random.rand(8, 3, 3, 1) - 0.5
    b1 = np.random.rand(8,1) - 0.5

    # Fully connected layers
    W2 = np.random.rand(1352, 64) - 0.5
    b2 = np.random.rand(64,1)-0.5
    W3 = np.random.rand(64, 10)-0.5
    b3 = np.random.rand(10,1)-0.5

    return W1, b1, W2, b2, W3, b3



def Flatten(X):
    return X.reshape(-1)

def ForwardProp(X, W1, b1, W2, b2, W3, b3):
    # First Convolution Layer
    Z1 = conv2D(X, W1) + b1.reshape(1, 1, 1, -1)
    A1 = ReLU(Z1)
    P1 = MaxPool(A1, pool_size=(2, 2))

    # Flattening the pooled output
    F = P1.reshape(P1.shape[0], -1)  # Flatten the output to match W3 dimensions

    # Fully Connected Layers
    Z2 = np.dot(F, W2) + b2 
    A2 = ReLU(Z2)

    Z3 = A2.dot(W3) + b3.T
    A3 = softmax(Z3)

    return Z1, A1, P1, Z2, A2, F, Z3, A3

def MaxPool_backward_multi_channel(A_prev, grad_out, pool_size=(2, 2), stride=2):
    print("A_prev.shape:", A_prev.shape)
    print("grad_out.shape:", grad_out.shape)
    (m, n_H_prev, n_W_prev, n_C_prev) = A_prev.shape
    (m, n_H, n_W, n_C) = grad_out.shape
    grad_input = np.zeros_like(A_prev)
    
    pool_height, pool_width = pool_size
    
    for b in range(m):  # Iterate over the batch dimension
        for h in range(n_H):  # Iterate over height dimension
            for w in range(n_W):  # Iterate over width dimension
                for c in range(n_C):  # Iterate over channels
                    # Define the window for max pooling
                    start_i = h * stride
                    start_j = w * stride
                    end_i = start_i + pool_height
                    end_j = start_j + pool_width
                    
                    # Ensure indices are within bounds
                    if end_i <= n_H_prev and end_j <= n_W_prev:
                        window = A_prev[b, start_i:end_i, start_j:end_j, c]
                        # Find the index of the maximum value in the window
                        #print("Window:", window)
                        max_idx = np.unravel_index(np.argmax(window), window.shape)
                        grad_input[b, start_i + max_idx[0], start_j + max_idx[1], c] = grad_out[b, h, w, c]
                    else:
                        # Handle edge cases where the pooling window might be out of bounds
                        grad_input[b, start_i:end_i, start_j:end_j, c] += grad_out[b, h, w, c]
    
    return grad_input

def backward_conv2D(dZ, X, W):
    """
    Perform backward convolution.
    Args:
        dZ: Gradients from the next layer, shape (batch_size, output_height, output_width, n_filters).
        X: Input to the forward pass, shape (batch_size, input_height, input_width, input_channels).
        W: Filters used in the forward pass, shape (n_filters, filter_height, filter_width, input_channels).
    Returns:
        dX: Gradient w.r.t input, shape same as X.
        dW: Gradient w.r.t weights, shape same as W.
        db: Gradient w.r.t biases, shape (n_filters, 1).
    """
    batch_size, output_height, output_width, n_filters = dZ.shape
    _, input_height, input_width, input_channels = X.shape
    n_filters, filter_height, filter_width, input_channels = W.shape

    # Initialize gradients
    dX = np.zeros_like(X)
    dW = np.zeros_like(W)
    db = np.zeros((n_filters, 1))  # Ensure db has shape (n_filters, 1)

    # Perform backward convolution
    for i in range(batch_size):
        for f in range(n_filters):
            for h in range(output_height):
                for w in range(output_width):
                    # Slice the region of input X that contributed to dZ
                    x_slice = X[i, h:h+filter_height, w:w+filter_width, :]
                    # Accumulate gradients for W
                    dW[f, :, :, :] += x_slice * dZ[i, h, w, f]
                    # Accumulate gradients for X
                    dX[i, h:h+filter_height, w:w+filter_width, :] += W[f, :, :, :] * dZ[i, h, w, f]
                    # Accumulate gradients for biases
                    db[f, 0] += dZ[i, h, w, f]  # Update db with shape (n_filters, 1)

    return dX, dW, db


def MaxPool_upsample(dA, A_prev, pool_size=(2, 2), stride=2):
    """
    Upsample the gradient from MaxPooling to match the input dimensions.
    """
    (m, n_H_prev, n_W_prev, n_C_prev) = A_prev.shape
    (m, n_H, n_W, n_C) = dA.shape
    upsampled = np.zeros_like(A_prev)
    
    pool_height, pool_width = pool_size
    
    for b in range(m):  # Iterate over the batch
        for h in range(n_H):  # Iterate over the height
            for w in range(n_W):  # Iterate over the width
                for c in range(n_C):  # Iterate over the channels
                    start_i = h * stride
                    start_j = w * stride
                    end_i = start_i + pool_height
                    end_j = start_j + pool_width
                    
                    window = A_prev[b, start_i:end_i, start_j:end_j, c]
                    max_idx = np.unravel_index(np.argmax(window), window.shape)
                    
                    upsampled[b, start_i + max_idx[0], start_j + max_idx[1], c] = dA[b, h, w, c]
    return upsampled


def BackwardProp(X, Y, Z1, A1, P1, Z2, A2, F, Z3, A3,W1, b1, W2, b2, W3, b3):
    m = X.shape[0]# number of batches
    dZ3 = A3 - one_hot(Y).T #(64,10)
    dW3 = 1/m * np.dot(A2.T,dZ3)#(64,10)
    db3 = 1/m * np.sum(dZ3,axis=0,keepdims=True).T#(10,1)

    dA2 = np.dot(dZ3,W3.T)#(64,64)
    dZ2 = np.dot(dA2,ReLU_deriv(Z2))#(64,64)
    dW2 = 1/m * np.dot(F.T,dZ2)#(1352,64)
    db2 = 1/m * np.sum(dZ2,axis=1,keepdims=True)#(64,1)

    dF = np.dot(dZ2,W2.T)#(64,1352)

    dP1 = dF.reshape(P1.shape)

    dA1 = MaxPool_upsample(dP1,A1)
    dZ1 = dA1 * ReLU_deriv(Z1)
    dX,dW1,db1 = backward_conv2D(dZ1,X,W1)
    dW1 = 1/m * dW1
    db1 = 1/m * db1
    return dW1, db1, dW2, db2, dW3, db3



def UpdateParams(W1, b1, W2, b2, W3, b3, dW1, db1, dW2, db2, dW3, db3, lr):
    # Update weights and biases using gradient descent
    #learning_rate:lr
    W1 -= lr * dW1
    b1 -= lr * db1
    W2 -= lr * dW2
    b2 -= lr * db2
    W3 -= lr * dW3
    b3 -= lr * db3
    
    return W1, b1, W2, b2, W3, b3

def Train(X,Y,epochs,batch_size,lr):
    W1, b1, W2, b2, W3, b3 = init_params()
    num_samples = X.shape[0]
    total_loss = 0
    count = 0
    for epoch in range(epochs+1):
        for X_batch,Y_batch in generate_batches(X,Y,batch_size):
            print(f"Count:{count}")
            Z1, A1, P1, Z2, A2, F, Z3, A3 = ForwardProp(X_batch,W1, b1, W2, b2, W3, b3)
            
            batch_loss = -np.mean(np.sum(one_hot(Y_batch).T * np.log(A3 + 1e-8), axis=1))
            total_loss += batch_loss
            
            dW1, db1, dW2, db2, dW3, db3 = BackwardProp(X_batch,Y_batch,Z1, A1, P1, Z2, A2, F, Z3, A3,W1, b1, W2, b2, W3, b3)
            W1, b1, W2, b2, W3, b3 = UpdateParams( W1, b1, W2, b2, W3, b3,dW1, db1, dW2, db2, dW3, db3,lr)
            count += 1
        avg_loss = total_loss / (num_samples // batch_size)
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {avg_loss:.4f}")

    return W1, b1, W2, b2, W3, b3


            

In [8]:
data = np.array(data)
m, n = data.shape
np.random.shuffle(data) # shuffle before splitting into dev and training sets

data_dev = data[0:1000]
Y_dev = data_dev[0]
X_dev = data_dev[1:n]
X_dev = X_dev / 255.

data_train = data[41936:m]
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train / 255.
_,m_train = X_train.shape

In [9]:
label = data_train[:,0]

In [10]:
label.shape

(64,)

In [11]:
images = data_train[:,1:]

In [12]:
images.shape

(64, 784)

In [13]:
images = images.reshape(-1, 28, 28, 1)  # Reshape to (num_samples, 28, 28, 1)
images = images/255.

In [14]:
images.shape

(64, 28, 28, 1)

In [15]:
W1, b1, W2, b2, W3, b3 = Train(X=images,Y=label,epochs=1,batch_size=64,lr=0.01)

Count:0
Epoch 1/1, Loss: 5.7593
Count:1
Epoch 2/1, Loss: 11.0329
